In [11]:
import torch
# from torch import nn
from torch.utils.data import DataLoader
# from torchvision import datasets
# from torchvision.transforms import ToTensor

import pandas as pd
from sklearn.model_selection import train_test_split

ImportError: DLL load failed while importing _C: The specified module could not be found.

# Prepare Data

In [4]:
df = pd.read_csv("./data/iris.csv")
df['Species'] = df['Species'].astype('category')
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
df['Species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

In [6]:
label_dict = dict(enumerate(df['Species'].cat.categories))
label_dict

{0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}

In [7]:
df['y'] = df['Species'].astype('category').cat.codes
df.drop(columns=['Id', 'Species'], inplace=True)
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,y
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   y              150 non-null    int8   
dtypes: float64(4), int8(1)
memory usage: 5.0 KB


In [9]:
TRAIN_SIZE, VALID_SIZE, TEST_SIZE = 0.6, 0.2, 0.2
train_df, tmp_df = train_test_split(df, train_size=TRAIN_SIZE)
valid_df, test_df = train_test_split(tmp_df, test_size=TEST_SIZE/(TEST_SIZE+VALID_SIZE))
train_df.shape[0], valid_df.shape[0], test_df.shape[0]

(90, 30, 30)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

AttributeError: module 'torch' has no attribute 'cuda'